We are working with three different models of the MNIST data from Kaggle. The notebook that I developed them in was getting crowded, so I decided to save the models and start fresh.

In [1]:
import numpy as np
from datetime import datetime, time, timedelta

import pandas as pd
import tensorflow as tf
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


import my_keras_utils as my_utils

In [5]:
model_1 = keras.models.load_model('MNIST_model.hdf5')
model_2 = keras.models.load_model('mnist_2.hdf5')
model_3 = keras.models.load_model('mnist_3.hdf5')

## Load our data.
## Since the load process is a little slow, the try-except allows us to re-run all 
## cells without having to wait. 
try:
    ## Raises NameError and loads data if X_train is not defined.
    X_train.shape
except NameError: 
    ((X_train, y_train), (X_dev, y_dev), (X_test, y_test)) = my_utils.load_kaggle_mnist(random_state=42)



In [3]:
adaptive_lr = keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    patience= 4,
                    cooldown= 8,
                    min_lr=.0003, 
                    factor=0.334,)

progress_update = my_utils.TimedProgressUpdate(2)

In [6]:
def cross_validate(model, X, y, val_size = 2000, folds = 3):
    init_weights = model.get_weights()
    results = []
    for fold in range(folds):
        X_val = X[fold*val_size:(fold+1)*val_size,:]
        X_train = np.concatenate((X[0:fold*val_size,:],X[(fold+1)*val_size:,:]))
        y_val = y[fold*val_size:(fold+1)*val_size]
        y_train = np.concatenate((y[0:fold*val_size],y[(fold+1)*val_size:]))
        history = model.fit(X_train, y_train,
            epochs=200,
            batch_size=128, 
            validation_data=(X_val, y_val),
            callbacks = [adaptive_lr, 
                        progress_update, 
                        ],
            verbose = 0)
        model.set_weights(init_weights)
        results.append(history)
    return results

results = []
for model in [model_1, model_2, model_3]:
    model.compile(optimizer = keras.optimizers.Adam(.003),
                    loss = "sparse_categorical_crossentropy",
                    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
    )
    histories = cross_validate(model, X_train, y_train)
    results.append(histories)

Begin training of functional_1 at 09:34:29. Progress updates every 120.0 seconds.
Starting training on  epoch 113. Current loss is 0.02810586802661419.
Finished fitting at 09:38:01. Elapsed time 0:03:31.390780.
Begin training of functional_1 at 09:38:01. Progress updates every 120.0 seconds.
Starting training on  epoch 114. Current loss is 0.0303637832403183.
Finished fitting at 09:41:32. Elapsed time 0:03:30.877064.
Begin training of functional_1 at 09:41:32. Progress updates every 120.0 seconds.
Starting training on  epoch 114. Current loss is 0.027229808270931244.
Finished fitting at 09:45:04. Elapsed time 0:03:32.140746.
Begin training of functional_1 at 09:45:05. Progress updates every 120.0 seconds.
Starting training on  epoch 116. Current loss is 0.03660748526453972.
Finished fitting at 09:48:32. Elapsed time 0:03:27.240559.
Begin training of functional_1 at 09:48:33. Progress updates every 120.0 seconds.
Starting training on  epoch 116. Current loss is 0.046205852180719376.
Fin

In [8]:
for e in results:
    for h in e:
        print(h.history['loss'][-1], h.history['val_loss'][-1])
    print()

0.026975354179739952 0.03466843441128731
0.0286782905459404 0.01121292170137167
0.02703363448381424 0.027117028832435608

0.0338074192404747 0.06785991042852402
0.03775924816727638 0.05883964151144028
0.03562828525900841 0.07905200123786926

0.0707157701253891 0.05916003882884979
0.07082317024469376 0.06924143433570862
0.06561960279941559 0.07626297324895859



In [10]:
model_1.evaluate(X_dev, y_dev)
model_2.evaluate(X_dev, y_dev)
model_3.evaluate(X_dev, y_dev)


63/63 [==============================] - 0s 1ms/step - loss: 2.3807 - acc: 0.0965


[2.3806636333465576, 0.09650000184774399]

In [ ]:
model_3.fit(X_train, y_train,
            epochs=200,
            batch_size=32, 
            validation_data=(X_val, y_val),
            callbacks = [adaptive_lr, 
                        progress_update, 
                        ],
            verbose = 0)

In [43]:
model_1.compile(optimizer = keras.optimizers.Adam(.003),
                    loss = "sparse_categorical_crossentropy",
                    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

model_1.fit(X_train, y_train,
            epochs=100,
            batch_size=32, 
            validation_data=(X_dev, y_dev),
            callbacks = [adaptive_lr, 
                        progress_update, 
                        ],
            verbose = 0)
model_1.evaluate(X_train, y_train)
model_1.evaluate(X_dev, y_dev)

Begin training of functional_1 at 11:52:53. Progress updates every 120.0 seconds.


KeyboardInterrupt: 

In [13]:
model_1.save("MNIST_model.hdf5")

In [33]:
comp_data = pd.read_csv('data/test.csv')
comp_data.index.name = 'ImageId'

In [27]:
predictions = model_1.predict(comp_data)

In [37]:
comp_data['Label'] = np.argmax(predictions, axis=1)
comp_data.index +=1

In [38]:
comp_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,Label
ImageId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [39]:
comp_data['Label'].to_csv('my_submission.csv')